# Few-shot learning

**Few-shot learning** é um paradigma de aprendizado de máquina no qual um modelo é treinado (ou ajustado) para realizar uma *tarefa de classificação* com **poucos exemplos rotulados por classe**. Tipicamente, entre 1 e 10 são usados. A ideia central é permitir que modelos generalizem bem, mesmo quando os dados rotulados são escassos, algo que humanos conseguem fazer com facilidade.

Comparação com outros paradigmas:

| Paradigma               | Nº de exemplos por classe | Exemplo típico                                       |
| ----------------------- | ------------------------- | ---------------------------------------------------- |
| **Supervised learning** | Centenas ou milhares      | Classificação de imagens com ImageNet                |
| **Few-shot learning**   | De 1 a \~10               | Identificar uma nova espécie de planta com 5 imagens |
| **Zero-shot learning**  | 0 (nenhum exemplo)        | LLMs respondendo perguntas sem fine-tuning           |
| **One-shot learning**   | 1 exemplo                 | Reconhecimento facial (ex: FaceNet)                  |

Na prática, few-shot learning geralmente envolve um modelo **pré-treinado** e algum mecanismo de **adaptação com poucos dados**. Algumas abordagens comuns incluem:

* **Meta-learning (aprendizado de como aprender):** o modelo é treinado em várias tarefas pequenas para aprender a se adaptar rapidamente a novas tarefas.

  * Ex: MAML (Model-Agnostic Meta-Learning)
* **Metric learning (aprendizado por distância):** aprende-se uma função de similaridade para comparar amostras (ex: prototypical networks, Siamese networks).
* **Prompting em LLMs:** em vez de re-treinar o modelo, fornecemos exemplos diretamente no prompt (ex: “Given these 3 examples, what comes next?”).

Como exemplo, imagine um LLM (e.g., Gemini, GPT-4, Llama) recebendo o seguinte prompt para classificar sentimentos:

```text
Exemplo 1: "Estou muito feliz com o atendimento!" → Sentimento: positivo  
Exemplo 2: "O produto chegou quebrado." → Sentimento: negativo  
Exemplo 3: "A entrega foi no prazo, mas o item não era o que esperava." → Sentimento: neutro  
Texto: "Gostei do produto, mas a entrega demorou." → Sentimento:
```

Esse é um exemplo de **few-shot prompting**, pois o modelo vê poucos exemplos antes de fazer a inferência.

In [3]:
%run get_llm.py

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable
from langchain_core.output_parsers import StrOutputParser

# Suponha que esta função já esteja definida em outro arquivo importado
# def get_llm(model_backend="ollama", model_name=None): ...

# Exemplo de prompt few-shot
prompt_template = PromptTemplate.from_template(
    """Classifique o sentimento das frases abaixo como positivo, negativo ou neutro.

Exemplos:
Frase: "Estou muito feliz com o atendimento!"  
Sentimento: positivo

Frase: "O produto chegou quebrado e ninguém responde meu e-mail."  
Sentimento: negativo

Frase: "A entrega foi no prazo, mas o item não era o que esperava."  
Sentimento: neutro

Agora, classifique a seguinte frase:
Frase: "{frase}"  
Sentimento:"""
)

# Constrói a cadeia com a nova API baseada em Runnable
llm = get_llm()
chain: Runnable = prompt_template | llm | StrOutputParser()

# Exemplo de uso
frase_teste = "O serviço foi bom, mas poderia ser mais rápido."
output = chain.invoke({"frase": frase_teste})
print("Sentimento classificado:", output.strip())

/home/ebezerra/ailab/gcc1734/notebooks/genai/get_llm.py:10: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  return ChatOllama(


Sentimento classificado: Sentimento: neutro 

**Justificativa:** A frase contém um elemento positivo ("o serviço foi bom") e um elemento negativo ("poderia ser mais rápido"), mas o tom geral é de avaliação, e não de forte emoção positiva ou negativa.
